# Dynamic selection of classiﬁers—A comprehensive review (Britto Jr)
<p class='lead'>
Author: Oliveira, Markos F. B. G.<br />
Date: 8/25/2017
</p>

# Description

In this notebook the algorithms described in the paper *Dynamic selection of classiﬁers—A comprehensive review* are implemented.

In [338]:
import numpy as np
import pandas as pd
import urllib
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#Wine data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
raw_data = urllib.request.urlopen(url)
dataset = np.loadtxt(raw_data, delimiter=",")
Xw = dataset[:, 1:]
yw = dataset[:, 0]

X_train, X_test, y_train, y_test = train_test_split(Xw, yw, stratify=yw, train_size=100)

### Individual-based measures

### (3.1.1) Ranking-based measures

#### (1) DSC-Rank method

To be done..

### (3.1.2) Accuracy-based measures

#### (2) DS-LA OLA-based method

In [168]:
from sklearn.neighbors import KNeighborsClassifier
def ds_la_ola(pool, X_val, y_val, X_test, k=3):
    """Returns the predicted values of new instances considering the local best classifier in a pool. If ties occur on
    determining the best local classifier, a majority voting is used.
    
    Parameters
    ----------
    pool : list of sklearn-estimators
        List of base-estimators
    X_val : np.ndarray
        Validation inputs.
    y_val : np.ndarray
        Validation outputs.
    X_test : np.ndarray
        Test inputs.
    k : int
        Neighborhood size.
    n_best : int
        Number of classifiers to keep.
    Returns
    -------
    y_pred : np.ndarray
        Predictions on the test set.
    """
    
    pool = np.array(pool)
    if X_test.ndim == 1:
        X_test = X_test.reshape(1,-1)
    
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_val, y_val)
    assert(k < X_val.shape[0])
    
    y_pred = [] #list of predictions for each test instance.
    for xi in X_test: #for each test instance.
        y_class = []
        for cl in pool: #for each classifier
            y_class.append(cl.predict([xi])[0])
        if y_class.count(y_class[0]) == len(y_class): #check if all classifiers return same label
            y_pred.append(y_class[0])
        else:
            ind = clf.kneighbors(X=[xi], n_neighbors=k, return_distance=False)
            X_k = np.take(X_val, ind, axis=0)[0] #data base of k nearest neighbors
            y_k = np.take(y_val, ind, axis=0)[0]
            model_scores = [] # list of tuples: (model, score)
            for cl in pool:#for each classifier
                model_scores.append((cl, cl.score(X_k, y_k)))
            scs = np.array([x[1] for x in model_scores])
            print(scs)
            print('opa2')
            print(dt1.predict([xi])[0])
            print(dt2.predict([xi])[0])
            print(dt3.predict([xi])[0])
            best_models_ids = np.argwhere(scs == np.amax(scs)) #select the indices of best models (>1 if ties)
            best_model_preds = []
            for best_cl in pool[np.array(best_models_ids)]:
                best_model_preds.append(best_cl[0].predict([xi])[0])
            if best_models_ids.shape == (1, 1): #no ties: cover the case where there is no tie between model performances
                #considering local validation points (X_l, y_l), i.e., only one model is better and thus, is used to make the
                #prediciton on xi.
                y_pred.append(pool[best_models_ids[0][0]].predict([xi])[0])
                print(pool[best_models_ids[0][0]].predict([xi])[0])
            elif best_model_preds.count(best_model_preds[0]) == len(best_model_preds): #no ties: cover the case where there is
                #tie bewteen different models performances considering the local validatoin points (X_l, y_l),
                #but the predictions of this models in the new instance is the same.
                y_pred.append(best_model_preds[0])
                print(best_model_preds[0])
            else: #ties
                print('tie')
                (values,counts) = np.unique(np.array(best_model_preds), return_counts=True)
                ind=np.argmax(counts)
                y_pred.append(values[ind])
                print(values[ind])        
    return np.array(y_pred)

Remarks:

- It's important to use validation points, not used in training to validate the local performances os each single classifier. Most classifiers can overfit the training data and could return very-high performances on local subspaces of the feature space. Using a set of points that were not used in training guarantee a unbiased proxy of the generalization error of each individual classifier on the subspace of the new instance.

- Ties are very often using this approach, maily for small k's: several classifiers can return the same neighbor validaiton error. In this case, the majority predicted class is returned considering the best classifiers on the k neighbors only. Note that even when using this approach, ties may remain. Suppose Pc1, Pc2 and Pc3 as performances on the k neighbors of classifiers C1, C2 and C3 respectively. In addition, consider the predicted classes as c1, c2 and c1. If (Pc1==Pc2) and (Pc1>Pc3) it's not clear which class we should return: c1 or c2. One approach whould be to use the probabilities predicted by C1 and C2 and choose the predicted class, whose classifier (among the best ones) has more confidence (higher probability).

- Majority voting may not be the best approach to decide the prediciton, when ties in the k-neigbors occur. One example that shwoed up in the trials are the following: three classifiers has local performances 0.4, 0.4, 0.4; i.e. a tie occured. The classes predicted were 2, 1, 1. The confidences for each prediciton were [0, 1, 0], [0.5, 0, 0.5] and [0.5, 0, 0.5] respectively. Because majority voting was used, class one was returned. It can be seen that even though the most common class were 1, the individual confidences were very low. Probably, a good approach to avoid this is to predict the class with the higher mean probability value. The integration parameter when ties occur could be selected by the user. Note however, when we perform integration, we loose the intuition of choosing a single classifier to perform some prediciton.

- Should we choose the classifier for an instance where the local performances (for all classifiers) are very low (below random guessing)?

- Only the best classifier is returned. It whould be nice to generalize this to N-best classifiers.

- How to use the distance between points to icnrease selection performance?

#### (3) DS-LA LA-based method

In [93]:
from sklearn.neighbors import KNeighborsClassifier
def ds_la_la(pool, X_val, y_val, X_test, k=5):
    """Returns the predicted values of new instances considering the local best classifier in a pool. If ties occur on
    determining the best local classifier, a majority voting is used.
    
    Parameters
    ----------
    pool : list of sklearn-estimators
        List of base-estimators
    X_val : np.ndarray
        Validation inputs.
    y_val : np.ndarray
        Validation outputs.
    X_test : np.ndarray
        Test inputs.
    k : int
        Neighborhood size.
    n_best : int
        Number of classifiers to keep.
    Returns
    -------
    y_pred : np.ndarray
        Predictions on the test set.
    """
    
    #A separate storage is done for each target class:
    knns = [] #stores tuples (class, base)
    for tgt in np.unique(y_val):
        #print('t:', X_val[[y_val==tgt]])
        clf = KNeighborsClassifier(n_neighbors=k).fit(X_val[[y_val==tgt]], y_val[[y_val==tgt]])
        #print(clf.kneighbors(X=[[-1,-1,-1]], n_neighbors=1, return_distance=False))
        knns.append((tgt, clf))
        assert(k <= X_val[[y_val==tgt]].shape[0])
    
    pool = np.array(pool)
    if X_test.ndim == 1:
        X_test = X_test.reshape(1,-1)
    
    y_pred = [] #list of predictions for each test instance.
    for xi in X_test: #for each test instance.
        y_class = []
        for cl in pool: #for each classifier
            y_class.append(cl.predict([xi])[0])
        if y_class.count(y_class[0]) == len(y_class): #check if all classifiers return same label
            y_pred.append(y_class[0])
        else:
            model_scores = [] # list of scores (sorted as classifiers are sorted in 'pool')
            for pred, cl in zip(y_class, pool): #for each predicted class of classifiers
                print('pred:', pred)
                knn = [tpl[1] for tpl in knns if tpl[0] == pred]
                ind = knn[0].kneighbors(X=[xi], n_neighbors=k, return_distance=False) #nearest neighbors of xi that has the
                #same labels returned by classifier cl.
                X_k = np.take(X_val[[y_val==pred]], ind, axis=0)[0] #data base of k nearest neighbors
                y_k = np.take(y_val[[y_val==pred]], ind, axis=0)[0]
                #print(X_k)
                model_scores.append((cl, cl.score(X_k, y_k)))
            scs = np.array([x[1] for x in model_scores])
            print('scores:',scs)
            print('opa2')
            print(dt1.predict([xi])[0])
            print(dt2.predict([xi])[0])
            print(dt3.predict([xi])[0])
            best_models_ids = np.argwhere(scs == np.amax(scs)) #select the indices of best models (>1 if ties)
            best_model_preds = []
            for best_cl in pool[np.array(best_models_ids)]:
                best_model_preds.append(best_cl[0].predict([xi])[0])
            if best_models_ids.shape == (1, 1): #no ties: cover the case where there is no tie between model performances
                #considering local validation points (X_l, y_l), i.e., only one model is better and thus, is used to make the
                #prediciton on xi.
                y_pred.append(pool[best_models_ids[0][0]].predict([xi])[0])
                print(pool[best_models_ids[0][0]].predict([xi])[0])
            elif best_model_preds.count(best_model_preds[0]) == len(best_model_preds): #no ties: cover the case where there is
                #tie bewteen different models performances considering the local validatoin points (X_l, y_l),
                #but the predictions of this models in the new instance is the same.
                y_pred.append(best_model_preds[0])
                print(best_model_preds[0])
            else: #ties
                print('tie')
                (values,counts) = np.unique(np.array(best_model_preds), return_counts=True)
                ind=np.argmax(counts)
                y_pred.append(values[ind])
                print(values[ind])        
    return np.array(y_pred)

Remarks:

- The same remarks pointed about OLA method makes sense here, because the high similarity between both merging methods.

### (3.1.3) Probabilistic-based measures

#### (4) A Priori/A Posteriori method

In [304]:
from sklearn.neighbors import KNeighborsClassifier
def a_priori_posteriori(pool, X_val, y_val, X_test, k=5, method='priori', threshold=0.1, low_conf='rnd'):
    """Returns the predicted values of new instances considering the local best classifier in a pool. If ties occur on
    determining the best local classifier, a majority voting is used.
    
    Parameters
    ----------
    pool : list of sklearn-estimators
        List of base-estimators
    X_val : np.ndarray
        Validation inputs.
    y_val : np.ndarray
        Validation outputs.
    X_test : np.ndarray
        Test inputs.
    k : int
        Neighborhood size.
    n_best : int
        Number of classifiers to keep.
    approach : str
        Method used to compute local accuracies. Valid values: 'priori', 'posteriori'.
    threshold: float
        Threshold used to compute the level of confidence of the selected classifier with maximum p_correct.
    low_conf: float
        Method used to predict the value of the new instance if the maximum p_correct is not above than the
        threshold of others. If 'rnd' is selected a random classifier with p_correct value higher than max p_correct - threshold
        is selected. If 'ola' the approch based on local accuracy proposed by Woods is used. Valid values: 'rnd', 'ola'.
    Returns
    -------
    y_pred : np.ndarray
        Predictions on the test set.
    """
    
    assert((method == 'priori') | (method == 'posteriori'))
    assert((low_conf == 'rnd') | (low_conf == 'ola'))
    
    #Full storage.
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_val, y_val)
    assert(k < X_val.shape[0])
    
    pool = np.array(pool)
    if X_test.ndim == 1:
        X_test = X_test.reshape(1,-1)
    
    y_pred = [] #list of predictions for each test instance.
    for instance, xi in enumerate(X_test): #for each test instance.
        #print('*********Instance:', instance)
        y_class = []
        for cl in pool: #for each classifier
            y_class.append(cl.predict([xi])[0])
        if y_class.count(y_class[0]) == len(y_class): #check if all classifiers return same label
            y_pred.append(y_class[0])
        else:
            
            ind = knn.kneighbors(X=[xi], n_neighbors=k, return_distance=False)
            X_k = np.take(X_val, ind, axis=0)[0] #data base of k nearest neighbors
            y_k = np.take(y_val, ind, axis=0)[0]
            
            p_corrects = [] # list of tuples": stores p_corrects for each classifier index in the pool.
            for i, (pred, cl) in enumerate(zip(y_class, pool)): #for class/predicted class
                
                #See steps at section 3.4 of Giacinto's paper 'Methods for Dynamic Classifier Selection'
                #STEP 1. Compute p_correct
                if method == 'priori':
                    nom = 0
                    deltas = []
                    for row_x, row_y in zip(X_k, y_k):
                        delta_i = 1/(np.linalg.norm([xi]-row_x))
                        deltas.append(delta_i)
                        itemindex = np.where(cl.classes_==row_y)[0][0]
                        P_hat_i = cl.predict_proba([row_x])[0][itemindex] #probability of current classifier (cl)
                        #predict the correct class of neighbor row_x, row_y.
                        nom = nom + P_hat_i*delta_i
                    p_correct = nom/sum(deltas)
                    
                else: #posteriori
                    nom = 0.0
                    den = 0.0
                    for row_x, row_y in zip(X_k, y_k):
                        delta_i = 1/(np.linalg.norm([xi]-row_x))
                        #print('Delta_i', delta_i)
                        itemindex = np.where(cl.classes_== pred)[0][0]
                        #print('Predict proba:', cl.predict_proba([row_x])[0][itemindex])
                        den = den + cl.predict_proba([row_x])[0][itemindex]*delta_i
                        if row_y == pred:
                            nom = nom + cl.predict_proba([row_x])[0][itemindex]*delta_i
                    p_correct = nom/den
                    #print('Nom:', nom)
                    #print('Den:', den)
                    #print('p_correct:', p_correct)
                    
                #STEP 2. Reject classifiers if p_correct<0.5
                if (p_correct < 0.5) | (p_correct != p_correct): #reject classifier. The second condition is for nan case.
                    continue #goes to the next classifier
                else:
                    p_corrects.append((i, p_correct))
            
            if not p_corrects: #if the list is empty use the best classifier in all validation set. Handle the cases where
                #the predict_proba for pred class (in the posteriori case) is 0 (which causes a p_correct == nan).
                print('Any p_correct > 0.5 was found!')
                scores = []
                for cl in pool:
                    scores.append(cl.score(X_val, y_val))
                y_out = pool[np.argmax(np.array(scores))].predict([xi])[0]
                y_pred.append(y_out)
                #print('Pred all validation:', y_out)
                continue #goes to the next instance    
            
            #STEP 3. Identify Cj with max p_correct
            p_corrects = sorted(p_corrects, key=lambda x: x[1]) #sort in ascending order
            p_max = p_corrects[-1][1] #if there is more than one pmax?
            c_max = pool[p_corrects[-1][0]]
            #print('p_corrects:', p_corrects)
            #print('p_max:', p_max)
            
            #STEP 4. Compute differences beween p_max and P_correct in p_corrects
            diff = [] #list of tuples (cl_index, difference)
            if method == 'priori':
                for tpl in p_corrects[:-1]: #removing only the max value.
                    dif = p_max - tpl[1]
                    diff.append((tpl[0], dif))
            else:#If 'a posteriori' method is selected, then the difference dj is evaluated only for the classifiers
            #that take a decision different from the one taken by c_max.
                c_max_pred = c_max.predict([xi])[0]
                #print('c_max_pred:', c_max_pred)
                for tpl in p_corrects[:-1]:
                    tpl_pred = pool[tpl[0]].predict([xi])[0]
                    #print('tpl_pred:', tpl_pred)
                    if not tpl_pred == c_max_pred:
                        dif = p_max - tpl[1]
                        diff.append((tpl[0], dif))
            #print('Diff:', diff)
            if not diff: #if diff is empty
                y_pred.append(c_max.predict([xi])[0])
                #print('Pred empty:', c_max.predict([xi])[0])
                continue
            #STEP. 5 Checks the 'confidence' of the selected c_max classifier
            if all(dif[1] > threshold for dif in diff):
                y_pred.append(c_max.predict([xi])[0]) #high confidence, c_max can be selected safely
                #print('Pred threshold:', c_max.predict([xi])[0])
            else: #low confidence: random selection of good classifiers
                
                ids = [tpl[0] for tpl in diff if tpl[1] < threshold] #indices of classifiers in p_corrects that have high values
                #(low difference), it does not include the c_max index.
                ids = np.array(ids)
                #print('Ids before append:', ids)
                ids = np.append(ids, -1) #adding the c_max index.
                #print('Ids after append:', ids)
                if low_conf == 'rnd':
                    random_id = np.random.choice(ids)
                    random_cl = pool[p_corrects[int(random_id)][0]]
                    y_pred.append(random_cl.predict([xi])[0])
                    #print('Pred random:', random_cl.predict([xi])[0])
                else:
                    #list of 'good'classifiers:
                    good_cl = []
                    for idd in ids:
                        good_cl.append(pool[p_corrects[idd][0]]) #at least two classifiers will be on this list.
                    y_out = ds_la_ola(good_cl, X_val, y_val, xi, k=k)
                    y_pred.append(y_out[0])
                    
    return np.array(y_pred)

Remarks:

- When randomly selecting the classifier with reasonable p_correct value, it would be possible to instead of using a uniform distribution of the classifiers, use a distribution based on the value itself: classifiers with higher p_correct values have more chance to be selected.

- It seems that's not a good idea to use the random strategy to choose a classifier, particularly in the 'a posteriori' case. It's not clear why should we use only the classifiers that take a decision different from the one in c_max. This may cause a lot of classifiers that have different predicted class than p_max; and in the random method, the c_max has little chance to win (even in the ola ensemble method). In addition, in the experiments was common to see only two remaining classifiers with different predicted labels. We should make a lot of effort to classify this instance as correct, becaue probably, such instance is a hard case, and classifing it correclty is what makes a successfull classification strategy.

In [305]:
#X_train = np.array([[0,0,0],[1,1,1],[2,2,2],[3,3,3],[4,4,4],[5,5,5],[6,6,6],[7,7,7],[8,8,8],[9,9,9]])
#y_train = np.array([0,1,0,1,0,1,0,1,0,1])
#X_test = np.array([[2.1,2.1,2.1],[-1,-1,-1]])
from sklearn.tree import DecisionTreeClassifier
#dt1 = DecisionTreeClassifier(max_depth=3)
#dt1.fit(X_train, y_train)
#dt2 = DecisionTreeClassifier(max_depth=2, random_state=9)
#dt2.fit(X_train, y_train)
#dt3 = DecisionTreeClassifier(max_depth=1, random_state=9)
#dt3.fit(X_train, y_train)
y = a_priori_posteriori([dt1, dt2, dt3], X_train, y_train, X_test, k=5, method = 'priori', low_conf='rnd')

### (3.1.4) Behavior-based measures

#### (5) DS-MCB method

In [335]:
from sklearn.neighbors import KNeighborsClassifier
def ds_mcb(pool, X_val, y_val, X_test, k=5, threshold=0.1, ola_threshold=0.1):
    """Returns the predicted values of new instances considering the local best classifier in a pool. If ties occur on
    determining the best local classifier, a majority voting is used.
    
    Parameters
    ----------
    pool : list of sklearn-estimators
        List of base-estimators
    X_val : np.ndarray
        Validation inputs.
    y_val : np.ndarray
        Validation outputs.
    X_test : np.ndarray
        Test inputs.
    k : int
        Neighborhood size.
    n_best : int
        Number of classifiers to keep.
    approach : str
        Method used to compute local accuracies. Valid values: 'priori', 'posteriori'.
    threshold: float
        Threshold used to define if a neighbor is similar to the new instance according to their predicitons.
    ola_threshold: float
        Threshold used to define if the classifier with maximum local accuracy is bigger enough to be used safely.
    Returns
    -------
    y_pred : np.ndarray
        Predictions on the test set.
    """
    
    #Full storage.
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_val, y_val)
    assert(k < X_val.shape[0])
    
    pool = np.array(pool)
    if X_test.ndim == 1:
        X_test = X_test.reshape(1,-1)
    
    y_pred = [] #list of predictions for each test instance.
    for instance, xi in enumerate(X_test): #for each test instance.
        #print('*********Instance:', instance)
        y_class = []
        for cl in pool: #for each classifier
            y_class.append(cl.predict([xi])[0])
        if y_class.count(y_class[0]) == len(y_class): #check if all classifiers return same label
            y_pred.append(y_class[0])
        else:
            mcb_t = y_class
            #Find nearest neighbors
            ind = knn.kneighbors(X=[xi], n_neighbors=k, return_distance=False)
            X_k = np.take(X_val, ind, axis=0)[0] #data base of k nearest neighbors
            y_k = np.take(y_val, ind, axis=0)[0]
            
            mcbs = [] #list of tuples. Each tuple has row_x, row_y and list of similarity. Number of internal tuples/lists =
            #number of neighbors; number of entries in each list = number of classifiers in the pool.
            for row_x, row_y in zip(X_k, y_k):
                mcb_n = []
                for cl in pool:
                    pd = cl.predict([row_x])[0]
                    mcb_n.append(pd)
                mcbs.append((row_x, row_y, mcb_n))
            
            sim_neighbors_x = []
            sim_neighbors_y = []
            while not sim_neighbors_x: #avoids 'sim_neighbors' being empty.
                for mcb_j in mcbs:
                    sim = np.mean(np.array([m==n for m, n in zip(mcb_j[2], mcb_t)]))
                    if sim > threshold:
                        sim_neighbors_x.append(mcb_j[0])
                        sim_neighbors_y.append(mcb_j[1])
                if not sim_neighbors_x: 
                    threshold = threshold + 0.1;
            
            scores = [] #list of tuples (cl_idx, score)
            for i, cl in enumerate(pool):
                Xv = np.array(sim_neighbors_x)
                yv = np.array(sim_neighbors_y)
                scores.append((i, cl.score(Xv, yv))) #local accuracy of neighbor with similar patterns than new instance
            
            scores = sorted(scores, key=lambda x: x[1])
            
            if all(sc[1] > scores[-1][1] - ola_threshold for sc in scores[:-1]): #scores[-1][1] is the maximum score
                ids = [sc[0] for sc in scores if sc[1] > scores[-1][1] - ola_threshold] #ids of classifiers with 'high' score.
                preds = []
                for idd in ids:
                    preds.append(pool[idd].predict([xi])[0])
                y_pred.append(np.bincount(preds).argmax()) #returns the most frequent prediction
            else: #high confidence in the max
                y_pred.append(pool[scores[-1][0]].predict([xi])[0])
                    
    return np.array(y_pred)

Remarks:
    
- The original paper states "If the classifier outputs can be regarded as estimates of the class posterior probabilities, these
probabilities can be taken into account in order to improve the estimation of CLA." A majority scheme was used in this algorithm, but it could be used the information of 'predict_proba' to enhance accuracy.

- A step not presented in the original algorithm was implemented. It iteratively increases the similarity threshold by 10% until neighbors with similar MCB are found. 

### (3.1.5) Oracle-based measures

#### (6) KNE method

To be done..